## Importing different packages

In [1]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import data, color
from google.colab import auth
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Dropout
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers, models, Input
import cv2
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input

### loading data from google drive

In [2]:
drive.mount('/content/drive', force_remount=True)
list_sub_dir = os.listdir('/content/drive/MyDrive/asl_dataset_1')

Mounted at /content/drive


In [3]:
len(list_sub_dir)

36

In [4]:
# list_item = os.listdir(f"/content/drive/MyDrive/asl_dataset_1/{list_sub_dir[0]}/")
# https://www.tensorflow.org/tutorials/images/classification

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory
PATH = '/content/drive/MyDrive/asl_dataset_1/'
nx = 100
ny = 100
train_dataset = image_dataset_from_directory(PATH,shuffle=True,color_mode="rgb",label_mode='categorical',validation_split=0.3,batch_size=64,labels='inferred',image_size=(nx,ny),subset="training",seed=123)
val_dataset = image_dataset_from_directory(PATH,shuffle=True,color_mode="rgb",label_mode='categorical',validation_split=0.3,batch_size=16,labels='inferred',image_size=(nx,ny),subset="validation",seed=123)

Found 2515 files belonging to 36 classes.
Using 1761 files for training.
Found 2515 files belonging to 36 classes.
Using 754 files for validation.


In [6]:
input_t    = Input(shape=(nx, ny, 3))
rescal     = layers.Rescaling(1./255)(input_t)
vgg_output = VGG16(weights="imagenet", include_top=False, input_shape=(nx,ny,3))(rescal)
base_model = Model(input_t, vgg_output)
base_model.trainable = False
base_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
Total params: 14,714,688
Trainable params: 0
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
# from tensorflow.data import AUTOTUNE
# AUTOTUNE = AUTOTUNE
# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(500, activation='relu')
prediction_layer = layers.Dense(36, activation='softmax')
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer,
    prediction_layer
])

In [9]:
from tensorflow.keras import optimizers
opt = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 500)               2304500   
                                                                 
 dense_1 (Dense)             (None, 36)                18036     
                                                                 
Total params: 17,037,224
Trainable params: 2,322,536
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1, restore_best_weights=True)

history = model.fit(x=train_dataset,  epochs=50, verbose=1, validation_data=val_dataset,
                 shuffle=True,  initial_epoch=0,callbacks=[es]) 

Epoch 1/50
28/28 [==============================] - 314s 11s/step - loss: 1.5958 - accuracy: 0.6139 - val_loss: 0.4206 - val_accuracy: 0.8992
Epoch 2/50
28/28 [==============================] - 7s 232ms/step - loss: 0.3234 - accuracy: 0.9074 - val_loss: 0.3201 - val_accuracy: 0.9019
Epoch 3/50
28/28 [==============================] - 7s 225ms/step - loss: 0.2094 - accuracy: 0.9313 - val_loss: 0.1658 - val_accuracy: 0.9576
Epoch 4/50
28/28 [==============================] - 7s 225ms/step - loss: 0.1079 - accuracy: 0.9705 - val_loss: 0.1245 - val_accuracy: 0.9655
Epoch 5/50
28/28 [==============================] - 7s 228ms/step - loss: 0.0830 - accuracy: 0.9824 - val_loss: 0.1089 - val_accuracy: 0.9788
Epoch 6/50
28/28 [==============================] - 7s 224ms/step - loss: 0.0620 - accuracy: 0.9858 - val_loss: 0.1274 - val_accuracy: 0.9629
Epoch 7/50
28/28 [==============================] - 7s 230ms/step - loss: 0.0671 - accuracy: 0.9824 - val_loss: 0.0907 - val_accuracy: 0.9735
Epoch 

In [11]:
base_model.trainable = False
opt = optimizers.Adam(learning_rate=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1, restore_best_weights=True)
history = model.fit(x=train_dataset,  epochs=50, verbose=1, validation_data=val_dataset,
                 shuffle=True,  initial_epoch=0,callbacks=[es]) 

Epoch 1/50
28/28 [==============================] - 8s 240ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.0512 - val_accuracy: 0.9894
Epoch 2/50
28/28 [==============================] - 7s 221ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0502 - val_accuracy: 0.9894
Epoch 3/50
28/28 [==============================] - 7s 223ms/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.0503 - val_accuracy: 0.9894
Epoch 4/50
28/28 [==============================] - 7s 222ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.0499 - val_accuracy: 0.9894
Epoch 5/50
28/28 [==============================] - 7s 223ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.0498 - val_accuracy: 0.9894
Epoch 6/50
28/28 [==============================] - 7s 224ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0497 - val_accuracy: 0.9894
Epoch 00006: early stopping


In [13]:
# class_names = train_dataset.class_names
# class_names

In [14]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_dataset.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")